# Hands-On Image Workshop: *Detecting Defects in Steel Strips Using Computer Vision* 

Within this hands-on workshop you will use a two different convolutional neural networks (CNNs) to classify different types of defects commonly found in steel strips. 

(And, if you're anything like the author you'll what on Earth defects in steel strips even look like!)

The steps which you will carry out are: 
1. Load the data
2. Perform some data exploration and visualisation
3. Build a couple of different CNNs and deploy these to Seldon
4. Add metadata to your newly created models
5. Create a drift detector and once again create this on Seldon

The model training code has been adapted from this blog post by `franky` on Medium: [Deep Learning, Computer Vision, and Automated Optical Inspection](https://towardsdatascience.com/deep-learning-computer-vision-and-automated-optical-inspection-774e8ca529d3)

Firstly, we will install and import the relevant packages which we will use throughout the exploration, training, and deployment process. Google Colab comes with a number of packages pre-installed, so we only need to install any additional packages we may need.

In [ ]:
!pip install seldon_deploy_sdk
!pip install alibi-detect==0.8.1
!pip install dill
!pip install tensorflow

In [ ]:
# General Packages
import os
import logging
import json
import numpy as np
import dill
import matplotlib.pyplot as plt
from functools import partial

# Model Building
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, Dense, Layer, Reshape, InputLayer, GlobalAveragePooling2D, Flatten
from tensorflow.keras import layers
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.models import Model

# Drift Detection
# from alibi_detect.cd import MMDDrift
# from alibi_detect.cd.tensorflow import preprocess_drift
# from alibi_detect.utils.saving import save_detector, load_detector

# Seldon Deploy SDK
from seldon_deploy_sdk import Configuration, ApiClient, SeldonDeploymentsApi, OutlierDetectorApi, DriftDetectorApi, ModelMetadataServiceApi
from seldon_deploy_sdk.auth import OIDCAuthenticator

# Logging and Clearing Session
tf.keras.backend.clear_session()
logger = tf.get_logger()
logger.setLevel(logging.ERROR)

Once the Python dependencies have been installed and imported you can download the training and testing data. This will take approximately 2 minutes. 

In [ ]:
!curl https://storage.googleapis.com/deploy-workshops/manufacturing/data/manufacturing-data.zip > manufacturing-data.zip
!mkdir data
!unzip -o 'manufacturing-data.zip' -d data

## Data Exploration

Loading the data into memory, as Keras `ImageDataGenerator` objects.

In [ ]:
train_datagen = ImageDataGenerator(rescale=1/255)
data_dir='data/train/images/'
train_ds = train_datagen.flow_from_directory(
    directory = data_dir,
    target_size = (224, 224),
    batch_size = 32,
    class_mode = 'categorical'
)

In [ ]:
val_datagen = ImageDataGenerator(rescale=1/255)
data_dir='data/validation/images/'
val_ds = train_datagen.flow_from_directory(
    directory = data_dir,
    target_size = (224, 224),
    batch_size = 32,
    class_mode = 'categorical'
)

The dataset has 6 classes which you can view below by loading them into the `categories` dictionary. 

In [ ]:
categories = (train_ds.class_indices)
categories = dict((v,k) for k,v in categories.items())
categories

You can readily visualise a sample image to start to gain an understanding of the data you are working with.

In [ ]:
example_image = train_ds.next()[0][0]
print(example_image.shape)
plt.imshow(example_image[:,:])

As you can see above; the images you'll be working with are 3 things: 
1. Relatively small, only 224 x 224 pixels. 
2. In colour as they have 3 channels, but appear greyscale. 
3. Pretty boring to look at!

The image below shows a wider variety of the types of image which are available within the dataset. 

![steel photo examples](https://raw.githubusercontent.com/SeldonIO/deploy-workshops/master/manufacturing/assets/steel_images.jpeg "Title")

## Building the Models

Now that the data is loaded in and you have had the chance to explore and get to grips with it, you can turn your attention to model building. 

You will build and work on 2 separate models: 
1. Training from scratch a simple hand crafted architecture, using 3 convolutional layers. 
2. Fine tuning an InceptionV3 model which has been pre-trained on the ImageNet dataset. 

Once these models have been created you are going to deploy them alongside one another as a canary deployment via the Seldon Deploy SDK. 

Prior to beginning the building efforts create a simple callback which will stop training early if the validation accuracy breaks 90%. 

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('val_accuracy') > 0.90 ):
            print("\nReached 90% validation accuracy so cancelling training!")
            self.model.stop_training = True 
            
callbacks = myCallback()

### Model 1: *Simple CNN*

Using Keras it is straightforward to define the CNN architecture, and this should be familiar if you have worked with CNNs before. 

Your neural network has three convolutional layers, each with 32 channels and a 3\*3 convolution. 

Looking at the model summary you can see the network has a total of approximately 11 million parameters. 

In [ ]:
simple_cnn = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)), # First Convolution
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'), # Second Convolution
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'), # Third Convolution
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(6, activation='softmax'),
])

simple_cnn.summary()

Training the network is then straightforward using standard methods; `categorical_crossentropy` for the loss function, and `adam` as the optimisation technique. 

In [ ]:
simple_cnn.compile(loss = 'categorical_crossentropy',
                   optimizer = 'adam',
                   metrics = ['accuracy'])

simple_cnn_history = simple_cnn.fit(train_ds,
                                    batch_size = 32,
                                    epochs = 3,
                                    validation_data = val_ds,
                                    callbacks = [callbacks],
                                    verbose = 1,
                                    shuffle=True)

Watching a neural network train is the modern equivalent of watching paint dry, therefore to speed things up feel free to interrupt training and grab the pre-trained example available below. 

In [ ]:
!mkdir simple-cnn/
simple_cnn.save('simple-cnn/')

In [ ]:

!gsutil cp -r simple-cnn/ gs://josh-seldon/workshops/manufacturing/pretrained/simple-cnn/1

In [ ]:
!gcloud auth login

In [ ]:
simple_cnn = load_model("simple-cnn/1")

Now that you have a trained neural network you can grab a test image and have your network generate a prediction!

In [ ]:
test_scratches = load_img('data/validation/images/scratches/scratches_241.jpg',
                          target_size=(224, 224, 3))

test_scratches = img_to_array(test_scratches)
test_scratches = test_scratches / 255
test_scratches  = test_scratches.reshape((-1,) + test_scratches.shape)

simple_preds = simple_cnn.predict(test_scratches)[0]

for v, i in enumerate(simple_preds):
    print(f"{categories[v]}: {i:.2f}")

As you can see the model is... **totally** wrong!

Even worse than that, it's confidently wrong- being fully convinced that the test scratch image has in fact been rolled in scale. 

The checkpointed model you're relying on here only acheived a validation accuracy of around 78% so this might be expected.

Let's see if this can be improved upon by using the InceptionV3 architecture.

### Model 2: *InceptionV3*

For your second model you will make use of the pre-trained InceptionV3 architecture. InceptionV3 is a CNN built for object classification by [researchers at Google in 2015](https://static.googleusercontent.com/media/research.google.com/en//pubs/archive/43022.pdf). Back then it held the state of the art crown for a while, however more recently has been overtaken by more modern EfficientNet or Transformer architectures. 

It has been selected here because it is reasonable to train on a CPU, it gets great results and Keras has a built-in version of the model making it super easy to get going with!

The key information to note around how you are preparing the InceptionV3 model is that when loaded you do not include the final layers (`include_top=False`). 

Three extra layers are then built on top of the InceptionV3 model; a pooling layer (`GlobalAveragePooling2D`), and two fully connected layers (`Dense`).

These new layers are glued onto InceptionV3 and compiled using the same evaluation methods as you used with the `simple-cnn`. 

In [ ]:
# parameters for architecture
input_shape = (224, 224, 3)
num_classes = 6
conv_size = 32

# parameters for training
batch_size = 32
num_epochs = 3

# load InceptionV3 from Keras
InceptionV3_model = InceptionV3(include_top=False, input_shape=input_shape)

# add custom Layers
x = InceptionV3_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation="relu")(x)
custom_output = Dense(num_classes, activation='softmax')(x)

# define the input and output of the model
inception = Model(inputs=InceptionV3_model.input, outputs=custom_output)
        
# compile the model
inception.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

inception.summary()

In [ ]:
inception_history = inception.fit(train_ds,
                                  batch_size=32,
                                  epochs=1,
                                  validation_data=val_ds,
                                  callbacks=[callbacks],
                                  verbose=1, 
                                  shuffle=True)

Watching an even larger neural network train is simply like watching an even larger wall dry, so once more you can take advantage of a pre-trained artefact.

In [ ]:
!mkdir inception/
inception.save('inception/')
!gsutil cp -r inception/ gs://josh-seldon/workshops/manufacturing/pretrained/inception/1 

In [ ]:
!mkdir inception/
!gsutil cp -r gs://josh-seldon/workshops/manufacturing/pretrained/inception/1 inception/

In [ ]:
inception = load_model("inception/1")

Similarly to the `simple_cnn` built earlier you can test your `inception` model on the same `test_scratch` image to get an anecdotal feel for how it's going to perform. 

In [36]:
inception_preds = inception.predict(test_scratches)[0]

for v, i in enumerate(inception_preds):
    print(f"{categories[v]}: {i:.2f}")

1/1 [==============================] - 1s 920ms/step
crazing: 0.00
inclusion: 0.00
patches: 1.00
pitted_surface: 0.00
rolled-in_scale: 0.00
scratches: 0.00


**Hurrah!** 

This result looks better, and you can also take some reassurance that this pre-trained `inception` model achieved a much more respectable validation accuracy of 94%. 

Time to start thinking about how you might want to serve your newly created models. 

## Deploying Your Models

Now that you have trained and saved your model artefacts you can begin to consider how to deploy them with Seldon. 

This process will involve uploading the relevant artefacts to blob storage, where they can be pulled from at deployment time. (As we will use the pretrained models rather than training in the workshop, the models are already in a google storage bucket, so we don't need to carry out the upload.)

You will then configure connection to a Seldon Deploy cluster, and leverage the pre-built Tensorflow Serving runtime to create your deployment. 

---

To begin with, you will push the `simple-cnn` and `inception` models to a Google storage bucket. It's worth noting that Seldon Deploy reads from a wide range of storage back ends and so all of the popular blob storage tools are covered as well. (As we will use the pretrained models rather than training in the workshop, the models are already in a google storage bucket, so we don't need to carry out the upload.).

You will create a unique folder with your name. Remember to replace `YOUR NAME` with your name.

Tensorflow Serving, similarly to Nvidia's Triton, when pulling model artefacts from a directory expects that there are sub-directories representing the version of the model artefacts. TF Serving can then be configured to serve a given model version. 

Given that you only have the single version for both of your models currently you will simply save version 1.

In [ ]:
# !gsutil cp -r simple-cnn/1/ gs://tom-seldon-examples/workshops/manufacturing/<YOUR NAME>/simple-cnn/
# !gsutil cp -r inception/1/ gs://tom-seldon-examples/workshops/manufacturing/<YOUR NAME>/inception/

## Creating Triton Config Files

In [ ]:
model_config = {
        "name": model_name,
        "platform": "tensorflow",
        "version_policy": {"specific": {"versions": [1]}},
        "input": [],
        "output": [],
    }

In [53]:
import tensorflow as tf
from tensorflow.python.saved_model import tag_constants
from tritonclient.utils import triton_to_np_dtype
from tritonclient.utils import np_to_triton_dtype
from google.protobuf import text_format
from tritonclient.grpc.model_config_pb2 import ModelConfig
from tensorflow.python.saved_model import saved_model_utils


def create_config(saved_model_dir, model_name):
    # Load the TensorFlow SavedModel
    loaded_model = tf.saved_model.load(saved_model_dir, tags=[tag_constants.SERVING])

    # Get the signature def from the loaded model
    signature_def = loaded_model.signatures[tf.saved_model.DEFAULT_SERVING_SIGNATURE_DEF_KEY]
    print(type(signature_def))
    # Create Triton model configuration
    model_config = ModelConfig()
    model_config.name = model_name
    model_config.platform = "tensorflow"

    # # Set the version policy (optional)
    # model_config.version_policy.max_versions = 1
    # model_config.

    # Iterate over the inputs in the signature def
    for input_name, input_tensor_info in signature_def.inputs.items():
        # Create Triton model input
        input_config = model_config.input.add()
        input_config.name = input_name
        input_config.data_type = triton_to_np_dtype(input_tensor_info.dtype)
        input_config.dims.extend(input_tensor_info.shape)

    # Iterate over the outputs in the signature def
    for output_name, output_tensor_info in signature_def.outputs.items():
        # Create Triton model output
        output_config = model_config.output.add()
        output_config.name = output_name
        output_config.data_type = triton_to_np_dtype(output_tensor_info.dtype)
        output_config.dims.extend(output_tensor_info.shape)

    # Save the Triton model config to a file
    with open("config.pbtxt", "w") as f:
        f.write(text_format.MessageToString(model_config))

# Usage
create_config("./simple-cnn", "simple-cnn")


<class 'tensorflow.python.saved_model.load._WrapperFunction'>


AttributeError: 'list' object has no attribute 'items'

In [55]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist
from tritonclient.utils import triton_to_np_dtype
from tritonclient.grpc.model_config_pb2 import ModelConfig
from google.protobuf import text_format

# Load and preprocess the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape((-1, 28, 28, 1)).astype("float32") / 255.0
x_test = x_test.reshape((-1, 28, 28, 1)).astype("float32") / 255.0

# Define the model architecture
model = tf.keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(10, activation='softmax')
])

# Compile and train the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=1, validation_data=(x_test, y_test))

# Save the trained model
model.save("trained_model")

# Create Triton model configuration
model_config = ModelConfig()
model_config.name = "mnist-model"
model_config.platform = "tensorflow"

# Set the version policy (optional)
# model_config.version_policy.max_versions = 1

# Iterate over the inputs in the model
input_config = model_config.input.add()
input_config.name = "input"
input_config.data_type = triton_to_np_dtype(x_train.dtype)
input_config.dims.extend(x_train[0].shape)

# Iterate over the outputs in the model
output_config = model_config.output.add()
output_config.name = "output"
output_config.data_type = triton_to_np_dtype(y_train.dtype)
output_config.dims.extend(y_train[0].shape)

# Save the Triton model config to a file
with open("config.pbtxt", "w") as f:
    f.write(text_format.MessageToString(model_config))


1875/1875 [==============================] - 7s 4ms/step - loss: 0.2043 - accuracy: 0.9422 - val_loss: 0.0802 - val_accuracy: 0.9753


TypeError: 'NoneType' object cannot be interpreted as an integer

In [48]:
create_config("./simple-cnn", "simple-cnn")

AttributeError: Protocol message Specific has no "version" field.

You can now deploy your models to the dedicated Seldon Deploy cluster which has been configured for this workshop. To do so you will interact with the Seldon Deploy SDK and deploy your model using that.

First, setting up the configuration and authentication required to access the cluster. Don't forget to update the `SD_IP` variable to your cluster IP.

In [56]:
SD_IP = "35.243.251.120"
config = Configuration()
config.host = f"https://{SD_IP}/seldon-deploy/api/v1alpha1"
config.oidc_client_id = "sd-api"
config.oidc_server = f"https://{SD_IP}/auth/realms/deploy-realm"
config.oidc_client_secret = "sd-api-secret"
config.auth_method = "client_credentials"
config.verify_ssl = False

def auth():
    auth = OIDCAuthenticator(config)
    config.id_token = auth.authenticate()
    api_client = ApiClient(configuration=config, authenticator=auth)
    return api_client

Now you have configured the IP correctly as well as setup the authentication function you can describe the deployment you would like to create. 

As you have uncovered earlier the `simple-cnn` is not great, and the `inception` model improves dramatically on that performance. This has been demonstrated on your validation data, but neither model has seen any live data. 

In this scenario you may wish to test both models on the live data, to achieve this Seldon Deploy provides you with two options; a shadow deployment or a canary deployment. 

These techniques are usually applied to the rollout or upgrade of a new model version. A shadow deployment deploys two models behind a single endpoint, both of the models receive 100% of the traffic- there is no split. However, only one of the models' responses are returned to users- usually the original model you are upgrading. The others responses (new version) are simply logged to Elasticsearch where you can see how it responded compared to the original model. This allows you to roll out new versions of a model and test them on live data, without the risk of poor performance impacting user experience. 

The other option is a canary deployment; this involves a traffic split between the two models behind a single endpoint. The percentage split is determined by you, and allows you to control how much of the live data goes to each of the models. New versions of models will typically be sent a smaller percentage of traffic (10% or 20%) to validate they are behaving as expected before they are rolled out more fully.

In this example you will create a canary deployment with a 70/30 split between the `inception` model and `simple-cnn` respectively. 

---

The [Seldon Deploy SDK](https://github.com/SeldonIO/seldon-deploy-sdk) offers two approaches to creating deployments: 
1. Working directly with dictionaries, which are equivalent to the underlying Kubernetes manifests. 
2. Using the higher level Python classes as abstractions. 

In this example you will use the more direct approach of working with the dictionaries. While this requires some more configuration it does create the opportunity to make reusable templates which are custom to your needs and can be readily shared and understood by others. 

You will start by defining some variables for the deployment which will be passed to the `mldeployment` manifest object in the following cell. 

In [57]:
YOUR_NAME = "jman"
DEPLOYMENT_NAME = f"{YOUR_NAME}-manufacturing"
NAMESPACE = "seldon-gitops"

MODEL_NAME = "inception"
MODEL_LOCATION = f"gs://josh-seldon/workshops/manufacturing/pretrained/simple-cnn/"

CANARY_NAME = "simple-cnn"
CANARY_LOCATION = f"gs://josh-seldon/workshops/manufacturing/pretrained/inception/"

CPU_REQUESTS = "1"
MEMORY_REQUESTS = "1Gi"

CPU_LIMITS = "1"
MEMORY_LIMITS = "1Gi"

In [58]:
mldeployment = {
    "kind": "SeldonDeployment",
    "metadata": {
        "name": DEPLOYMENT_NAME,
        "namespace": NAMESPACE
    },
    "apiVersion": "machinelearning.seldon.io/v1alpha2",
    "spec": {
        "name": DEPLOYMENT_NAME,
        "protocol": "seldon",
        "predictors": [
            {
                "componentSpecs": [
                    {
                        "spec": {
                            "containers": [
                                {
                                    "name": f"{CANARY_NAME}",
                                    "resources": {
                                        "requests": {
                                            "cpu": CPU_REQUESTS,
                                            "memory": MEMORY_REQUESTS
                                        },
                                        "limits": {
                                            "cpu": CPU_LIMITS,
                                            "memory": MEMORY_LIMITS
                                        }
                                    }
                                }
                            ]
                        }
                    }
                ],
                "name": "default",
                "replicas": 1,
                "traffic": 100,
                "graph": {
                    "implementation": "TENSORFLOW_SERVER",
                    "modelUri": CANARY_LOCATION,
                    "name": f"{CANARY_NAME}",
                    "logger": {
                        "mode": "all"
                    }
                }
            }
             ]
    }
}

Once you have specified your `mldeployment` JSON it is a simple API call to create your deployment. 

In [59]:
deployment_api = SeldonDeploymentsApi(auth())
deployment_api.create_seldon_deployment(namespace=NAMESPACE, mldeployment=mldeployment)

{'api_version': 'machinelearning.seldon.io/v1',
 'kind': 'SeldonDeployment',
 'metadata': {'annotations': None,
              'cluster_name': None,
              'creation_timestamp': None,
              'deletion_grace_period_seconds': None,
              'deletion_timestamp': None,
              'finalizers': None,
              'generate_name': None,
              'generation': None,
              'labels': None,
              'managed_fields': None,
              'name': 'jman-manufacturing',
              'namespace': 'seldon-gitops',
              'owner_references': None,
              'resource_version': None,
              'self_link': None,
              'uid': None},
 'spec': {'annotations': None,
          'name': 'jman-manufacturing',
          'oauth_key': None,
          'oauth_secret': None,
          'predictors': [{'annotations': None,
                          'component_specs': [{'hpa_spec': None,
                                               'keda_spec': None,
   

You can now test your model endpoint in the UI. To access the workshop cluster use the following details: 

Don't forget to replace XXXXX in the below URL with your cluster IP.

* Seldon Deploy URL: http://XXXXX/seldon-deploy/
* Username: admin@seldon.io
* Password: 12341234

Once you have seen your deployment creating you can save a sample image using the `test-scratches` array you worked with earlier. 

This `test-scratches.json` can be uploaded directly to the UI to test your model endpoint when it is available!

In [ ]:
seldon_sample = {
  "data": {
    "names": [
    ],
    "ndarray": test_scratches.tolist()
  }
}

with open('test-scratches.json', 'w', encoding='utf-8') as f:
    json.dump(seldon_sample, f, ensure_ascii=False, indent=1)

## Adding Model Metadata

You have now created 2 new models within a single deployment. Both of these models have been automatically added to the model catalog. The model catalog acts as a centralised repository for metadata associated with models. Models can be easily deployed directly from the catalog, while metadata acts to speed knowledge transfer between teams and to track models across tools.

You will now add some metadata to your InceptionV3 model describing the validation accuracy you managed to achieve, and who authored the model- so you can brag to your colleagues!

In [ ]:
model_catalog_metadata = {
      "URI": MODEL_LOCATION,
      "name": "InceptionV3",
      "version": "v1.0",
      "artifactType": "TENSORFLOW",
      "taskType": "defect classification",
      "tags": {
        "auto_created": "true",
        "author": f"{YOUR_NAME}"
      },
      "metrics": {},
      "creationTime": "2022-02-15T15:26:26.630592Z",
      "project": "default"
    }

In [ ]:
api_instance = ModelMetadataServiceApi(auth())
api_response = api_instance.model_metadata_service_update_model_metadata(model_catalog_metadata)

Listing the metadata for your newly updated model:

In [ ]:
api_response = api_instance.model_metadata_service_list_model_metadata(uri=MODEL_LOCATION)
api_response

# Drift Detection

The final stage of the deployment process is going to be adding the advanced monitoring capabilities afforded by a drift detector. 

In this example you will use Alibi Detect to train a custom drift detector which can flag when the underlying input data distribution has shifted. This can inform decisions about re-training or prompt deeper investigation into data/model behaviours. 

Seldon Deploy also allows you to setup alerts when drift is detected. 

---

In this example you will use the Maximum Mean Discrepancy method. Covariate or input drift detection relies on creating a distance measure between two distributions; a reference distribution and a new distribution. The MMD drift detector is no different; the mean embeddings of your features are used to generate the distributions and then the distance between them is measured. The training data is used to calculate the reference distribution, while the new distribution comes from your inference data.  

More technically, a reproducing kernel Hilbert space is used to generate the mean embeddings, by mapping the highly complex feature space within which most machine learning models operate to a linear Euclidean space. A radial basis function kernel is then used to measure the distance between the two embeddings, and the signifiance of the drift is calculated as a p-value using permutation/resampling tests. More details can be found [here](https://docs.seldon.io/projects/alibi-detect/en/v0.10.3/cd/methods/mmddrift.html).

Before you dive into creating your own drift detector, you need to generate a reference dataset. In this case you use 5 batches of the training data set, resulting in 160 images. This has been picked for convenience and speed of training in the workshop, and if this was a production case you would likely want to use the entirety of the training set, or a statistically significant segment.

Now, creating your reference set. 

In [ ]:
train_ds.reset()

drift_ref = train_ds.next()[0]

for i in range(4):
    drift_ref = np.concatenate((drift_ref, train_ds.next()[0]))

drift_ref.shape

Due to the high dimensionality of image data you will need perform a pre-processing step to make it computationally feasible to detect drift on incoming batches of images. 

A great option for this is to use an simple encoding neural network. This will squish images from their original shape of `(224, 224, 3)` to a vector of `32`. From there we can feed the vectors to the drift detector and calculate drift values. 

This is then used to a generate a `preprocess_fn` which will be fed to your MMD drift detector to convert your images ahead of time. 

In [ ]:
# define encoder
encoding_dim = 32
encoder_net = tf.keras.Sequential(
  [
      InputLayer(input_shape=(224, 224, 3)),
      Conv2D(64, 4, strides=2, padding='same', activation=tf.nn.relu),
      Conv2D(128, 4, strides=2, padding='same', activation=tf.nn.relu),
      Conv2D(512, 4, strides=2, padding='same', activation=tf.nn.relu),
      Flatten(),
      Dense(encoding_dim,)
  ]
)

# define preprocessing function
preprocess_fn = partial(preprocess_drift, model=encoder_net, batch_size=160)

Next you can initialise the drift detector; this is as simple as a single line API call specifying the:
* Reference data
* Computational backend
* The p-value below which drift is considered to have occurred
* Your preprocessing function
* The number of permutations taken to calculate the p-value

In [ ]:
# initialise drift detector
cd = MMDDrift(drift_ref, backend='tensorflow', p_val=.05, preprocess_fn=preprocess_fn, n_permutations=100)

Once your shiny new drift detector has trained you can test it on a new batch of data!

In [ ]:
new_batch = train_ds.next()[0]
preds = cd.predict(new_batch, return_p_val=True, return_distance=True)
preds

Finally, you can save the drift detector and upload it to Google storage for deployment. 

In [ ]:
filepath = 'defect-drift'
save_detector(cd, filepath)

In [ ]:
!gsutil cp -r defect-drift gs://tom-seldon-examples/workshops/manufacturing/{YOUR_NAME}/defect-drift

To deploy the drift detector you will use Seldon Deploy's user interface. Simply navigate to your deployment, and select the "Create" button for your drift detector.

This will bring up a form. Add your `Detector Name`, the `Storage URI`:


In [ ]:
print(f"gs://tom-seldon-examples/workshops/manufacturing/{YOUR_NAME}/defect-drift")

the `Batch Size`:

20

and the `Drift Type`:

Batch

The batch size configuration sets how many data points have to be sent to the endpoint before drift is calculated.

In order to test your drift detector, you can run the same image through your deployed model 20 times. You can see both in the drift detector logs and in the Drift Detection screen within the monitoring tab, that drift has occurred. 